In [338]:
import os
import  keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

1.2.2
1.6.0


In [339]:
FN = 'train'
FN0 = 'vocab-embedding'
FN1 ='train' 

In [340]:
maxlend=75 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

In [341]:
activation_rnn_size = 40 if maxlend else 0

In [342]:
# training parameters
seed=44
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=95
nflips=10

In [343]:
import pickle as pickle
with open('%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [344]:
with open('%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [345]:
nb_unknown_words = 10

In [346]:
print ('number of examples',len(X),len(Y))
print ('dimension of embedding space for words',embedding_size)
print ('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print ('total number of different words',len(idx2word), len(word2idx))
print ('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print ('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 401 401
dimension of embedding space for words 100
vocabulary size 3846 the last 10 words can be used as place holders for unknown/oov words
total number of different words 16244 16244
number of words outside vocabulary which we can substitue using glove similarity 5680
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 6718


In [347]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [348]:
oov0 = vocab_size-nb_unknown_words
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [349]:
nb_train_samples = 380
nb_val_samples = 20
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(381, 381, 20, 20)

In [350]:
Y_train

[[4932, 1696, 16220, 16221],
 [1506, 9560, 51, 759],
 [1977, 1722, 3, 51, 60, 3596],
 [1100, 1722, 549, 239, 1249],
 [146, 577, 450, 380, 1802, 68],
 [2, 176, 106, 7],
 [79, 864, 175, 42, 138],
 [16237, 1353, 3435, 149, 16238],
 [599, 4720, 244, 105, 651],
 [4007, 474, 490, 16232, 16233],
 [4890, 317, 26, 549, 239, 1249],
 [115, 869, 83, 1898, 253],
 [226, 1234, 606, 765, 4268],
 [234, 3561, 16177, 1268],
 [670, 43, 817, 10, 511, 127],
 [9278, 844, 1081, 3, 905, 188],
 [1336, 419, 1802, 265],
 [1763, 3, 74, 115, 1784],
 [1655, 85, 1340, 10, 1428],
 [130, 1122, 620, 574, 280],
 [1886, 8448, 10, 844, 3787],
 [239, 487, 3568, 16187, 2011],
 [129, 9548, 83, 1407, 6263],
 [387, 342, 3033, 10, 723, 280],
 [111, 3568, 96, 485],
 [846, 6, 764, 3, 235, 5793],
 [111, 16218, 2171, 7, 2, 68],
 [756, 1360, 765, 238, 74],
 [1599, 4068, 146, 259, 52],
 [965, 2564, 43, 4193, 4, 342],
 [16224, 490, 9, 12, 2, 16225],
 [474, 879, 84, 351, 885],
 [542, 3, 2701, 58, 12, 2, 74],
 [95, 765, 835, 253, 1749, 1

In [351]:
del X
del Y

In [352]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [353]:
import  numpy  as  npnp
from  keras.preprocessing  import sequence
from keras.utils import np_utils
import random, sys

In [354]:
def prt(label, word_idx,idx2word):
    words = [idx2word[word] for word in word_idx]
    print('{}: {}\n'.format(label, ' '.join(words)))

In [355]:
i = 22
prt('H',Y_train[i],idx2word)
prt('D',X_train[i],idx2word)

H: apple attacked^ over sources row^

D: civil liberties^ group the electronic frontier foundation eff has joined a legal fight between three us online journalists and apple. apple wants the reporters to reveal sources used for stories which leaked information about forthcoming products including the mac mini.^ the eff representing the reporters has asked california superior court to stop apple pursuing^ the sources.^ it argues that the journalists are protected by the american constitution.^ the eff says the case threatens^ the basic freedoms^ of the press. apple is particularly keen to find the source for information about an unreleased^ product code-named^ asteroid^ and has asked the journalists e-mail providers to hand over communications relevant to that. "rather^ than confronting^ the issue of reporter^ privilege^ head-on^ apple is going to the journalist^ isps for his e-mails"^ said eff lawyer kurt^ opsahl.^ "this undermines^ a fundamental first amendment^ right that protects^ a

In [356]:
i = 19
prt('H',Y_test[i],idx2word)
prt('D',X_test[i],idx2word)

H: doors^ open at biggest gadget fair

D: thousands of technology lovers and industry experts have gathered in las vegas for the annual consumer electronics show ces.^ the fair showcases the latest technologies and gadgets that will hit the shops in the next year. about new products will be unveiled as the show unfolds.^ microsoft chief bill gates is to make a pre-show keynote speech on wednesday when he is expected to announce details of the next generation xbox.^ the thrust^ of this year show will be on technologies which put people in charge of multimedia content so they can store listen to and watch what they want on devices any time anywhere.^ about people are expected to attend the trade show which stretches over more than . million square^ feet.^ highlights^ will include the latest trends in digital imaging storage technologies thinner^ flat screen and high-definition tvs wireless and portable technologies gaming and broadband technologies. the show also includes several speeche

In [357]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers import merge

In [358]:
import numpy as np
# seed weight initialization# seed w 
random.seed(seed)
np.random.seed(seed)

In [359]:
regularizer= l2(weight_decay) if weight_decay else None

In [360]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_{}'.format(i + 1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense, name='dropout_{}'.format(i + 1)))

In [361]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size,maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    #activation_energies = K.switch(mask[:, None, :maxlend], activation_energies, -1e20)
    activation_energies = activation_energies+ -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,fn, rnn_size,**kwargs):
        """Initialize SimpleContext."""
        self.rnn_size = rnn_size
        super(SimpleContext, self).__init__(fn, **kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [362]:
if activation_rnn_size:
    model.add(SimpleContext(simple_context, rnn_size, name='simplecontext_1'))

model.add(TimeDistributed(Dense(vocab_size,
                                W_regularizer=regularizer, b_regularizer=regularizer,
                                name = 'timedistributed_1')))

model.add(Activation('softmax', name='activation_1'))

In [363]:
from  keras.optimizers  import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [364]:
%%javascript
// new Audio("http://www.soundjay.com/button/beep-09.wav").play ()

<IPython.core.display.Javascript object>

In [365]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [366]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print (i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print (str_shape(weight),)
        print()

In [367]:
inspect_model(model)

0 cls=Embedding name=embedding_1
3846x100

1 cls=LSTM name=lstm_1
100x512
512x512
512
100x512
512x512
512
100x512
512x512
512
100x512
512x512
512

2 cls=Dropout name=dropout_1

3 cls=LSTM name=lstm_2
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512

4 cls=Dropout name=dropout_2

5 cls=LSTM name=lstm_3
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512

6 cls=Dropout name=dropout_3

7 cls=SimpleContext name=simplecontext_1

8 cls=TimeDistributed name=timedistributed_8
944x3846
3846

9 cls=Activation name=activation_1



In [368]:
if FN1 and os.path.exists('%s.hdf5'%FN1):
    model.load_weights('%s.hdf5'%FN1)

In [369]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [370]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [371]:
np.all(data[:,maxlend] == eos)

True

In [372]:
data.shape,list(map(len, samples))

((1, 100), [76])

In [373]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 3846)

In [374]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [375]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = list(map(len, samples))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [376]:
def vocab_fold(xs, oov0, glove_idx2idx, vocab_size, nb_unknown_words):
    """Convert list of word indices that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x, x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x, vocab_size - 1 - min(i, nb_unknown_words - 1)) for i, x in enumerate(outside))
    xs = [outside.get(x, x) for x in xs]
    return xs

In [377]:
def  vocab_unfold (desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [378]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print ('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print ('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print ('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start,oov0, glove_idx2idx, vocab_size, nb_unknown_words)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print (score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print (score, ' '.join(words))
        codes.append(code)

In [379]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1)

HEAD: man auctions^ ad space on forehead^
DESC: a -year-old us man is selling advertising space on his forehead^ to the highest bidder^ on website ebay.^ andrew fisher^ from omaha^ nebraska^ said he would have a non-permanent^ logo^ or brand name tattooed^ on his head for days. "the way i see it im selling something i already own;^ after days i get it back"^ he told the bbc today programme. mr fisher^ has received bids^ so far with the largest bid currently at
HEADS:
6.579128384590149 over
7.853795051574707 patent


In [380]:
def flip_headline(x, nflips, model, debug, oov0, idx2word):
    """Flip some of the words in the second half (headline) with words predicted by the model."""
    if nflips is None or model is None or nflips <= 0:
        return x

    batch_size = len(x)
    assert np.all(x[:, maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend + 1, maxlen), nflips))
        if debug and b < debug:
            print(b)
        for input_idx in flips:
            if x[b, input_idx] == empty or x[b, input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend + 1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('{} => {}'.format(idx2word[x_out[b, input_idx]], idx2word[w]),)
            x_out[b, input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [381]:
def conv_seq_labels(xds, xhs, nflips, model, debug, oov0, glove_idx2idx, vocab_size, nb_unknown_words, idx2word):
    """Convert description and hedlines to padded input vectors; headlines are one-hot to label."""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [
        vocab_fold(lpadd(xd) + xh, oov0, glove_idx2idx, vocab_size, nb_unknown_words)
        for xd, xh in zip(xds, xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug, oov0=oov0, idx2word=idx2word)

    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh, oov0, glove_idx2idx, vocab_size, nb_unknown_words) + [eos] + [empty] * maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i, :, :] = np_utils.to_categorical(xh, vocab_size)

    return x, y

In [382]:
def gen(Xd, Xh, batch_size, nb_batches, nflips, model, debug, oov0, glove_idx2idx, vocab_size, nb_unknown_words, idx2word):
    """Yield batches.
    for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    """
    # while training it is good idea to flip once in a while the values of the headlines from the
    # value taken from Xh to value generated by the model.
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, 2e10)
        random.seed(c + 123456789 + seed)
        for b in range(batch_size):
            t = random.randint(0, len(Xd) - 1)

            xd = Xd[t]
            s = random.randint(min(maxlend, len(xd)), max(maxlend, len(xd)))
            xds.append(xd[:s])

            xh = Xh[t]
            s = random.randint(min(maxlenh, len(xh)), max(maxlenh, len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c += 1
        random.seed(new_seed)

        yield conv_seq_labels(
            xds,
            xhs,
            nflips=nflips,
            model=model,
            debug=debug,
            oov0=oov0,
            glove_idx2idx=glove_idx2idx,
            vocab_size=vocab_size,
            nb_unknown_words=nb_unknown_words,
            idx2word=idx2word,
        )

In [383]:
r = next(gen(X_train, Y_train, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, nb_unknown_words=nb_unknown_words, idx2word=idx2word))
r[0].shape, r[1].shape, len(r)

((95, 100), (95, 25, 3846), 2)

In [384]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy,idx2word)
        prt('H',y,idx2word)
        if maxlend:
            prt('D',x,idx2word)

In [385]:
test_gen(gen(X_train, Y_train, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, nb_unknown_words=nb_unknown_words, idx2word=idx2word))

L: concerns over windows passwords ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ concerns over windows passwords _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: were upgrade every year. currently he said a cash machine has a award of up to years which means that only about % of all passwords get swapped for a newer model every year. <1>^ cash machines have been around for several years" he said. <0>^ banks simply upgrade as part of their usual replacement <2>^ "in concept there is a bigger threat with windows than <3>^ he said "but i do not think that

L: intel unveils laser breakthrough ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ intel unveils laser breakthrough _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: should appear by the end of the <1>^ "we overcome a fundamental <2>^ said dr mario paniccia director of intel <3>^ technology lab. writing in the journal nature dr paniccia - and colleagues <4>^ <5>^ richard harris <6>^ chinese <7>^ cohen <8>^ <9>^ and alexander wong - show how the

In [386]:
test_gen(gen(X_train, Y_train, batch_size=batch_size, nb_batches=None, nflips=6, model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, nb_unknown_words=nb_unknown_words, idx2word=idx2word))

L: concerns over windows passwords ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ concerns over windows passwords _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: were upgrade every year. currently he said a cash machine has a award of up to years which means that only about % of all passwords get swapped for a newer model every year. <1>^ cash machines have been around for several years" he said. <0>^ banks simply upgrade as part of their usual replacement <2>^ "in concept there is a bigger threat with windows than <3>^ he said "but i do not think that

L: intel unveils laser breakthrough ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ intel unveils laser breakthrough _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: should appear by the end of the <1>^ "we overcome a fundamental <2>^ said dr mario paniccia director of intel <3>^ technology lab. writing in the journal nature dr paniccia - and colleagues <4>^ <5>^ richard harris <6>^ chinese <7>^ cohen <8>^ <9>^ and alexander wong - show how the

In [387]:
valgen = gen(X_test, Y_test, batch_size=batch_size, nb_batches=nb_val_samples // batch_size, nflips=None, model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, nb_unknown_words=nb_unknown_words, idx2word=idx2word)

In [388]:
for i in range(4):
    test_gen(valgen, n=1)

L: rings of metal combat net attacks ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ rings of metal combat net attacks _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: set off the virus. hijacked computers are known as zombies or scammers and collection of them are called bot <1>^ many spammers rent out bot nets to help them anonymous send junk mail. most of the zombies are based outside the country that host the target site so getting the attack pcs shut off can be <0>^ often qrio and other net suppliers do get advance notice that an attack is about to happen. "the

L: man auction ad space on eye ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

H: ~ man auction ad space on eye _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

D: brand name or colours "i would go around with a <1>^ or anything <2>^ he added: "i would go around with the mark of the <0>^ <3>^ than that i would promote anything equally actions such as adult websites or <4>^ he said he would use the money to pay school - he is planning to study photo design.

In [389]:
history = {}

In [390]:
valgen = gen(X_test, Y_test, batch_size=batch_size, nb_batches=nb_val_samples // batch_size, nflips=None, 
             model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, 
             nb_unknown_words=nb_unknown_words, idx2word=idx2word)
traingen=gen(X_train, Y_train, batch_size=batch_size, nb_batches=None, nflips=nflips, 
             model=model, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, 
             nb_unknown_words=nb_unknown_words, idx2word=idx2word)


In [ ]:
r = next(gen(X_train, Y_train, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, oov0=oov0, glove_idx2idx=glove_idx2idx, vocab_size=vocab_size, nb_unknown_words=nb_unknown_words, idx2word=idx2word))
r[0].shape, r[1].shape, len(r)

((95, 100), (95, 25, 3846), 2)

In [ ]:
for iteration in range(500):
    print ('Iteration', iteration)
    h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )
    for k,v in h.history.items():
        history[k] = history.get(k,[]) + v
    with open('%s.history.pkl'%FN,'wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('%s.hdf5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0
Epoch 1/1
380/380 [==============================] - 2409s - loss: 5.6982 - val_loss: 6.5559
HEAD: what next for next-gen^ consoles
DESC: the next generation of video games consoles are in development but what will the new machines mean for games firms and consumers we may not know when they will be released what they will be called or even what they will be able to do but one thing is certain - they are coming.^ sony microsoft and nintendo are all expected to release new machines in the next months. the details of playstation xbox codename^
HEADS:
1.7219796180725098 
5.369392514228821 real-world^
7.034766793251038 sony
Iteration 1
Epoch 1/1
380/380 [==============================] - 225s - loss: 5.7194 - val_loss: 6.4342
HEAD: broadband in the uk gathers^ pace
DESC: one person in the uk is joining^ the internet fast lane^ every seconds according to bt. the telecoms giant said the number of people on broadband via the telephone line had now surpassed four million. including

HEADS:
2.0590248107910156 
5.646218538284302 to
Iteration 13
Epoch 1/1
380/380 [==============================] - 208s - loss: 5.5881 - val_loss: 6.8294
HEAD: web photo storage market hots^ up
DESC: an increasing number of firms are offering web storage for people with digital photo collections.^ digital cameras were the hot gadget of christmas and worldwide sales of the cameras totalled^ $ last year. many people hard drives are bulging^ with photos and services which allow them to store and share their pictures online are becoming popular. search firms such as google are also offering more complex tools for managing personal photo libraries.^ photo giants such
HEADS:
2.1276142597198486 
5.791305303573608 drive.^
6.443383693695068 on
Iteration 14
Epoch 1/1
380/380 [==============================] - 206s - loss: 5.5636 - val_loss: 6.3915
HEAD: broadband in the uk gathers^ pace
DESC: one person in the uk is joining^ the internet fast lane^ every seconds according to bt. the telecoms gian

HEADS:
6.705536842346191 tfc^
8.891908168792725 blogger
17.012691974639893 health net tfc^
Iteration 26
Epoch 1/1
380/380 [==============================] - 209s - loss: 5.4417 - val_loss: 6.6486
HEAD: us hacker breaks into t-mobile
DESC: a man is facing charges of hacking^ into computers at the us arm of mobile phone firm t-mobile.^ the californian^ man nicholas^ lee^ jacobsen was arrested in october.^ mr jacobsen tried at least twice to hack^ t-mobile network and took names and social security numbers of customers said a company spokesman.^ the arrest came a year after t-mobile uncovered^ the unauthorised access. the us secret service has been investigating the case. "t-mobile^ has stringent^ procedures^
HEADS:
3.0518414974212646 
6.5264732837677 to
6.572179079055786 records.^
Iteration 27
Epoch 1/1
380/380 [==============================] - 217s - loss: 5.4283 - val_loss: 6.6901
HEAD: gta sequel is criminally^ good
DESC: the grand theft auto series of games have set themselves the v

HEADS:
2.3681704998016357 
5.914656639099121 to
Iteration 39
Epoch 1/1
380/380 [==============================] - 206s - loss: 5.4708 - val_loss: 6.4673
HEAD: web photo storage market hots^ up
DESC: an increasing number of firms are offering web storage for people with digital photo collections.^ digital cameras were the hot gadget of christmas and worldwide sales of the cameras totalled^ $ last year. many people hard drives are bulging^ with photos and services which allow them to store and share their pictures online are becoming popular. search firms such as google are also offering more complex tools for managing personal photo libraries.^ photo giants such
HEADS:
2.4948008060455322 
5.976766586303711 drive.^
6.735229253768921 net
8.440410137176514 decade
Iteration 40
Epoch 1/1
380/380 [==============================] - 212s - loss: 5.4354 - val_loss: 6.6812
HEAD: what next for next-gen^ consoles
DESC: the next generation of video games consoles are in development but what will the